In [2]:
import os
import pandas as pd
df = pd.read_csv(os.path.join('commit_data.csv'))
pr_df = pd.read_csv(os.path.join('pr.csv'))
issue_df = pd.read_csv(os.path.join('issue.csv'))

In [3]:
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2024, 7, 6)

COMMITS

In [4]:
df['Updated_at'] = pd.to_datetime(df['Updated_at'])
commit_counts = df.groupby(df['Updated_at'].dt.date).size()
commit_counts = commit_counts.reset_index(name = 'Commit_count')
commit_counts

KeyError: 'Updated_at'

Các biểu đồ tổng quát

In [ ]:
sns.histplot(commit_counts["Commit_count"], color = 'green',kde=True)
plt.title("Commit count vs Count(Commit count)")
plt.show()

In [ ]:
sns.kdeplot(data=commit_counts['Commit_count'], color = 'red')
plt.title('Density_commits')
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
sns.lineplot(data=commit_counts, x='Updated_at', y='Commit_count')
plt.title("Number of Commits Over Days")
plt.xlabel('Updated at')
plt.ylabel("Number of Commits")
plt.show()

CONTRIBUTORS

Tổng số contributors đã đóng góp cho repo

In [ ]:
df['Contributor'].nunique()

Người đóng góp nhiều commit nhất và % trên tổng số commit

In [ ]:
max_commits = df['contributor'].value_counts().head(1)
len(df.index)
percentageContribution = max_commits/len(df.index)
print(percentageContribution)

contributor
NamedUser(login="alextran1502")    0.395809
Name: count, dtype: float64


Thời gian hoạt động của các contributor

In [ ]:
df['date'] = pd.to_datetime(df['date'])
times = df.groupby('author')['date'].agg(['min', 'max'])
print(times)

                                   min                 max
author                                                    
Alex               2022-02-08 05:42:35 2024-06-03 21:00:20
Alex The Bot       2023-05-09 02:08:01 2024-04-05 02:39:51
Alex Tran          2022-02-07 21:06:30 2024-03-07 18:04:54
Alexandre Bouijoux 2023-09-08 06:48:39 2023-09-08 06:48:39
Andrew Roberts     2024-03-08 19:05:15 2024-03-08 19:05:15
...                                ...                 ...
shenlong           2023-10-07 10:17:50 2024-02-08 03:07:43
twitsforbrains     2023-03-14 19:12:42 2023-03-14 19:12:42
vovchyk bratyk     2024-05-07 20:17:21 2024-05-07 20:17:21
waclaw66           2023-08-10 13:32:01 2023-08-10 13:32:01
wittymap           2023-06-05 13:56:39 2023-06-05 13:56:39

[126 rows x 2 columns]


Top 5 người commit nhiều nhất

In [ ]:
commit_count = df['author'].value_counts()
commit_count.head(5)

Top 5 người commit nhiều file nhất

In [ ]:
df_temp = df.sort_values('filename', ascending = False)
df_temp.head(5)

Top 5 người thay đổi nhiều file nhất trong 1 lần

In [ ]:
df_temp = df.sort_values('files changed', ascending = False)
df_temp.head(5)

Top 5 người có nhiều từ nhất trong commit message

In [ ]:
df_temp = df.sort_values('additions', ascending = False)
df_temp.head(5)

top 5 người có số lần all checks passed cao nhất và tỉ lệ all checks passed/ số lần commit

In [ ]:
top_all_checks_passed = df[df["all_checks_passed"]]
top_all_checks_passed = top_all_checks_passed['author'].value_counts()
list_tmp = top_all_checks_passed.head(5)
list_tmp

In [ ]:
df_temp = pd.merge(top_all_checks_passed, commit_count, on="author", how="inner").head(5)
df_temp['scale'] = df_temp['count_x']/df_temp['count_y']
df_temp

top 5 đuôi email nhiều người dùng nhất

In [ ]:
df['email_types'] = df['email'].str.extract(r'@([\w\.-]+)', expand=False)
df['email_types'].value_counts()

Người hoạt động đều đặn nhất- người có thời gian đóng góp dài nhất

ISSUES

Tổng số issues đã được mở và đóng góp trong repo này:

In [ ]:
issue_df.shape[0]

Thời gian trung bình để một issue được giải quyết

In [ ]:
issue_df['duration'] = pd.to_datetime(issue_df['Closed_at'])-pd.to_datetime(issue_df['Created_at'])
issue_df['duration'].mean()

In [ ]:
issue_df['State'].value_counts() 

PULL REQUEST

Tổng số pull request đã được tạo ra và merge

In [ ]:
pr_df.shape[0]

In [ ]:
len(pr_df[pr_df['Merged_at'].notnull()])

pull request đã bị reject

In [ ]:
len(pr_df[(pr_df['Merged_at'].isnull()) & (pr_df['Pr_state'] != 'open')])

LANGUAGES

Ngôn ngữ lập trình được sử dụng

Vẽ biểu đồ pie thể hiện tỷ lệ các ngôn ngữ lập trình được sử dụng trong repo này

FILES

Các files trong repo có nhiều commits nhất

In [ ]:
df_exploded = df.explode('filename')
file_counts = df_exploded['filename'].value_counts()
file_counts

Kích thước trung bình của các files

Tăng trưởng repo

Theo số lượng commits và contributors

Nhận xét các giai đoạn nổi bật

Dự đoán

Phân tích mạng lưới cộng tác: Vẽ mạng lưới (network graph) thể hiện sự cộng tác giữa các contributors. Ai là người có nhiều kết nối nhất

VISUALIZATION

Relationships between commits and contributors

In [ ]:
commit_contributor = pd.DataFrame()
commit_contributor['commits'] = df['author'].value_counts()
commit_contributor

In [ ]:
plt.figure(figsize=(50, 20))
colors = sns.color_palette("pastel", len(df['author'].value_counts().head(10))) 
sns.countplot(data=df, x='author', order=df['author'].value_counts().head(10).index, palette=colors)
plt.xticks(rotation=45, ha='right', fontsize=40) 
plt.yticks(fontsize=40) 
plt.xlabel('Author', fontsize=70)
plt.ylabel('Count', fontsize=70) 
plt.show()

In [ ]:
so.Plot(df,"additions", edgestyle='').scale(x='log').add(so.Bar(), so.Hist())

In [ ]:
pr_df['Rejected'] = (pr_df['Merged_at'].isnull()) & (pr_df['Pr_state'] != 'open')
sns.scatterplot(data=pr_df, x='Commits', y='Addititions', hue='Rejected', style = 'Rejected')